Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [15]:
from random import choices, randint, random, uniform, sample
from math import exp

import lab9_lib

In [16]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    # here, we print the the genome (as a string) and the fitness function of this genome
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")
    # fitness(ind) passes the genome using the __call__ function
print(fitness.calls)

01011010111001000010101001111011001000000101011101: 29.56%
00100000000010000101111011100010100000011001101001: 7.34%
01001111010001100101010100001110001111110010100100: 23.56%
11011110111110101101111111010100010101100110011010: 15.33%
01101100111010101101001001111111110001111001101010: 23.33%
10110011011001110100100111000101001110001010011010: 15.33%
01111110010010010001011100001111100100100101111111: 31.34%
11001011011011011100000001001000111101110001111011: 7.33%
00110000001110110000111110001111110010001110110011: 9.11%
00010010011001111000010010110100000111111000010001: 15.36%
10


## Local Search Algorithm
This Local Search utilizes the best individual obtained from the Genetic Algorithm. Initially, I set up the Local Search variables. During the algorithm's execution, neighbors are generated by randomly flipping a subset of bits.

In [17]:
class LocalSearch:
    def __init__(self, problem, initial_solution, iterations=10):
        self.problem = problem
        self.genome_size = len(initial_solution)
        self.best_individual = initial_solution
        self.best_fitness = problem(self.best_individual)
        self.iterations_without_improvement = 0
        self.iterations = iterations

    def run(self):
        for _ in range(self.iterations):
            neighbor = self.generate_neighbor(self.best_individual)
            neighbor_fitness = self.problem(neighbor)

            if neighbor_fitness <= self.best_fitness:
                self.iterations_without_improvement += 1
            else:
                self.best_individual = neighbor
                self.best_fitness = neighbor_fitness
                self.iterations_without_improvement = 0

            if self.best_fitness == 1.0:
                break

    def generate_neighbor(self, individual):
        index_to_mutate = choices(range(self.genome_size), k=self.genome_size)
        neighbor = list(individual)

        # randomly flipping of bits
        mutation_rate = 0.9  # mutation rate of 90%
        for index in index_to_mutate:
            if random() < mutation_rate:
                neighbor[index] = 1 - neighbor[index]

        return neighbor

    @staticmethod
    def acceptance_probability(current_fitness, new_fitness, temperature):
        delta_fitness = new_fitness - current_fitness
        if delta_fitness < 0:
            return 1.0
        else:
            return exp(-delta_fitness / temperature)

## Genetic Algorithm
Here, I define the true genetic algorithm.

In [18]:
class GeneticAlgorithm:
    def __init__(self, problem, genome_size=1000, population_size=50, generations=100, mutation_rate=0.1, local_search_iterations=10, convergence_threshold=5):
        # Setting up Genetic Algorithm parameters
        self.problem = problem
        self.genome_size = genome_size
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.local_search_iterations = local_search_iterations
        self.convergence_threshold = convergence_threshold

    def run(self):
        best_individual_after_ls = None
        count_no_improvement_steps = 0

        # Population initialization
        pop = [choices([0, 1], k=self.genome_size) for _ in range(self.population_size)]

        for _ in range(self.generations):
            # Fitness evaluation
            scores = [self.problem(individual) for individual in pop]
            best_individual = pop[scores.index(max(scores))]

            # Applying local search to the best individual
            ls = LocalSearch(self.problem, best_individual, iterations=self.local_search_iterations)
            ls.run()
            best_individual_after_ls = ls.best_individual
            fitness_after_local = ls.best_fitness

            # Checking for fitness improvement after local search
            if fitness_after_local <= self.problem(best_individual):
                count_no_improvement_steps += 1
                index_to_replace = scores.index(min(scores))
                pop[index_to_replace] = best_individual_after_ls
            else:
                count_no_improvement_steps = 0

            # Termination condition based on consecutive generations with no improvement
            if count_no_improvement_steps >= self.convergence_threshold:
                break

            # Updating population using crossover and mutation
            new_p = self.crossover(pop)
            new_p = self.mutate(new_p)
            pop = new_p

        # Returning the best individual after all generations
        return best_individual_after_ls

    def crossover(self, population):
        new_p = []

        for _ in range(self.population_size // 2):
            # Tournament selection of parents
            p_1 = choices(population)[0]
            p_2 = choices(population)[0]

            # Two-point crossover
            crossover_points = sorted([randint(0, self.genome_size - 1) for _ in range(2)])

            c_1 = p_1[:crossover_points[0]] + p_2[crossover_points[0]:crossover_points[1]] + p_1[crossover_points[1]:]
            c_2 = p_2[:crossover_points[0]] + p_1[crossover_points[0]:crossover_points[1]] + p_2[crossover_points[1]:]

            new_p.extend([c_1, c_2])

        return new_p
    
    

    def mutate(self, population):
        mutated_population = []
        
        for individual in population:
            mutated_individual = list(individual)  # Assuming each individual is represented as a list
        
            for i in range(len(mutated_individual)):
                if random() < self.mutation_rate:
                    # Mutate the gene with a random value
                    mutated_individual[i] = uniform(0, 1)  
            
            mutated_population.append(mutated_individual)
        
        return mutated_population



Here there is the main part, in which I call the genetic algorithm and I print all the results.

In [19]:
# Usage of the Genetic Algorithm
instances = [1, 2, 5, 10]

for instance in instances:
    fitness_value = instance
    fitness = lab9_lib.make_problem(fitness_value)
    fitness.calls_increment = fitness_value

    gen_alg = GeneticAlgorithm(problem=fitness, genome_size=1000, population_size=100, generations=50, mutation_rate=0.9, local_search_iterations=50, convergence_threshold=5)
    best_solution = gen_alg.run()

    print(f"Instance {instance}")
    print(f"Best fitness value: {fitness(best_solution):.2%}")

Instance 1
Best fitness value: 100.00%
Instance 2
Best fitness value: 100.00%
Instance 5
Best fitness value: 100.00%
Instance 10
Best fitness value: 100.00%
